# 02a - Cylinder Segmentation Demo

This notebook demonstrates mesh segmentation using a simple cylinder geometry. The cylinder represents the simplest case with genus = 0 (no holes).

**Part of the GenCoMo Demo Series** - [Return to Index](01_tutorial_index.ipynb)

In [ ]:
import numpy as np
import trimesh
from gencomo.demos import create_cylinder_mesh
from gencomo.segmentation import MeshSegmenter
from gencomo.visualization import visualize_mesh_3d

## Create and Visualize Cylinder

In [ ]:
# Create a cylinder mesh for testing
radius = 1.0
length = 2.0
cylinder = create_cylinder_mesh(radius=radius, length=length, resolution=32)

print(f"Cylinder properties:")
print(f"  Volume: {cylinder.volume:.3f} (theoretical: {np.pi * radius**2 * length:.3f})")
print(f"  Surface area: {cylinder.area:.3f} (theoretical: {2 * np.pi * radius * (radius + length):.3f})")
print(f"  Z-bounds: [{cylinder.bounds[0, 2]:.1f}, {cylinder.bounds[1, 2]:.1f}]")

# Visualize the original cylinder
fig = visualize_mesh_3d(cylinder, title="Original Cylinder", backend="plotly")
fig.show()

## Segment the Mesh

In [ ]:
# Create segmenter and segment the mesh
slice_width = 0.5
segmenter = MeshSegmenter()
segments = segmenter.segment_mesh(cylinder, slice_width=slice_width, min_volume=0.01)

print(f"Segmentation complete!")
print(f"Total segments: {len(segments)}")
print(f"Total slices: {len(segmenter.slices)}")

## Analyze Segmentation Results

In [ ]:
# Show segments per slice
print("Segments per slice:")
for i in range(len(segmenter.slices)):
    slice_segments = segmenter.get_segments_in_slice(i)
    print(f"  Slice {i}: {len(slice_segments)} segments")

# Get statistics
stats = segmenter.compute_segmentation_statistics()
print(f"\nSegmentation statistics:")
print(f"  Total volume: {stats['volume_stats']['total']:.4f}")
print(f"  Mean segment volume: {stats['volume_stats']['mean']:.4f}")
print(f"  Connected components: {stats['connected_components']}")

In [ ]:
# Detailed segment analysis
print("Detailed segment properties:")
print("-" * 80)

for i, seg in enumerate(segments):
    # Determine segment type based on position
    is_end_segment = seg.slice_index == 0 or seg.slice_index == len(segmenter.slices) - 1
    segment_type = "End" if is_end_segment else "Middle"
    
    print(f"Segment {i+1} ({segment_type}):")
    print(f"  ID: {seg.id}")
    print(f"  Slice Index: {seg.slice_index}")
    print(f"  Volume: {seg.volume:.4f}")
    print(f"  Exterior Surface Area: {seg.exterior_surface_area:.4f}")
    print(f"  Interior Surface Area: {seg.interior_surface_area:.4f}")
    print(f"  Centroid: [{seg.centroid[0]:.2f}, {seg.centroid[1]:.2f}, {seg.centroid[2]:.2f}]")
    print(f"  Z Range: [{seg.z_min:.3f}, {seg.z_max:.3f}] (width: {seg.z_max - seg.z_min:.3f})")
    print()

## Theoretical Validation

In [ ]:
# Validate segmentation against theoretical values
print("=== THEORETICAL VALIDATION ===")

# Theoretical calculations for r=1, h=2, slice_width=0.5
r = radius
h = length
expected_segments = int(h / slice_width)

# Expected values per segment
expected_volume_per_segment = np.pi * r**2 * slice_width  # π * r² * slice_width
expected_end_interior = np.pi * r**2  # One circular cut face
expected_end_exterior = np.pi * r**2 + 2 * np.pi * r * slice_width  # End cap + side
expected_middle_interior = 2 * np.pi * r**2  # Two circular cut faces
expected_middle_exterior = 2 * np.pi * r * slice_width  # Side area only

print(f"Expected values:")
print(f"  Number of segments: {expected_segments}")
print(f"  Volume per segment: {expected_volume_per_segment:.4f}")
print(f"  End segments - Interior: {expected_end_interior:.4f}, Exterior: {expected_end_exterior:.4f}")
print(f"  Middle segments - Interior: {expected_middle_interior:.4f}, Exterior: {expected_middle_exterior:.4f}")
print()

# Check volume conservation
total_volume_segments = sum(seg.volume for seg in segments)
volume_error = abs(total_volume_segments - cylinder.volume) / cylinder.volume * 100

print(f"Volume conservation:")
print(f"  Original cylinder volume: {cylinder.volume:.4f}")
print(f"  Sum of segment volumes: {total_volume_segments:.4f}")
print(f"  Conservation error: {volume_error:.2f}%")

# Check individual segment accuracy
print(f"\nIndividual segment validation:")
all_segments_valid = True

for i, seg in enumerate(segments):
    is_end = seg.slice_index == 0 or seg.slice_index == len(segmenter.slices) - 1
    expected_interior = expected_end_interior if is_end else expected_middle_interior
    expected_exterior = expected_end_exterior if is_end else expected_middle_exterior
    
    volume_error = abs(seg.volume - expected_volume_per_segment) / expected_volume_per_segment * 100
    interior_error = abs(seg.interior_surface_area - expected_interior) / expected_interior * 100
    exterior_error = abs(seg.exterior_surface_area - expected_exterior) / expected_exterior * 100
    
    segment_valid = volume_error < 5 and interior_error < 10 and exterior_error < 10
    status = "✅" if segment_valid else "❌"
    
    print(f"  Segment {i+1}: Vol {volume_error:.1f}%, Int {interior_error:.1f}%, Ext {exterior_error:.1f}% {status}")
    
    if not segment_valid:
        all_segments_valid = False

# Overall assessment
print(f"\n=== VALIDATION SUMMARY ===")
if volume_error < 1 and all_segments_valid and len(segments) == expected_segments:
    print("✅ SEGMENTATION ALGORITHM VALIDATION PASSED")
    print("   Volume conservation: EXCELLENT")
    print("   Surface area classification: CORRECT")
    print("   Individual segment accuracy: GOOD")
else:
    print("❌ SEGMENTATION ALGORITHM VALIDATION FAILED")
    if volume_error >= 1:
        print(f"   Volume conservation error: {volume_error:.2f}% (should be < 1%)")
    if not all_segments_valid:
        print("   Some segments have accuracy issues")
    if len(segments) != expected_segments:
        print(f"   Wrong number of segments: got {len(segments)}, expected {expected_segments}")

## Visualize Connectivity

In [ ]:
# Visualize connectivity in 3D space
try:
    segmenter.visualize_connectivity_graph_3d(backend="plotly")
except Exception as e:
    print(f"Visualization not available: {e}")
    print("You can install plotly for 3D visualization: pip install plotly")

## Summary

The cylinder segmentation demonstrates:

- **Simple topology**: Each slice contains one segment (genus = 0)
- **Linear connectivity**: Segments connect sequentially along the z-axis  
- **Accurate volume conservation**: Total volume is preserved during segmentation
- **Correct surface area classification**: Interior (cut) vs exterior (original) surfaces
- **Predictable geometry**: Results match theoretical calculations

This represents the baseline case for mesh segmentation and validates the core algorithm functionality.